### Khai báo thư viện

In [1]:
import pandas as pd
import re

### Tải dữ liệu

In [2]:
data = pd.read_excel('./Dataset/Nha_Thuoc.xlsx')
data.head(5)

,LOAIHINHHN,PHAMVIHN,TENCOSO,DUONG,PHUONG,QUAN,CVXLCCHN,SOCCHN,NGCAPCCHN,NOICAPCCHN,SODDK,SOGCN,GPP_GDP_GSP,NGAYCAPGCN,NGAYHHGCN
0,Đại lý bán thuốc của doanh nghiệp,Bán lẻ thuốc thành phẩm thuộc danh mục thuốc t...,Đại lý số 24 - Công ty TNHH một thành viên Dượ...,"C5/22, Đường Hưng Nhơn, Ấp 3",Xã Tân Kiên,Huyện Bình Chánh,Ds Trương Việt Cường,01531/HCM-CCHND,20/02/2014,Sở Y tế TP.HCM,6427/HCM-ĐKKDD,NaN,NaN,NaN,NaN
1,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Hộ kinh doanh Nhà thuốc Kim Châu,"9, Đường số 5, Khu phố 2",Phường Linh Xuân,Quận Thủ Đức,Ds. Trương Việt Cường,4524/CCHN-D-SYT-HCM,14/06/2019,Sở Y tế TP.HCM,7969/ĐKKDD-HCM,9807.0,GPP,18/03/2020,NaN
2,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Nhà thuốc Phương Hiền,"493, Man Thiện, Khu phố 5",Phường Tăng Nhơn Phú A,Quận 9,NaN,2306/ĐNAI-CCHND,13/07/2016,Sở Y tế Tỉnh Đồng Nai,1660/ĐKKDD-HCM,7816.0,GPP,01/12/2017,01/12/2020
3,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Địa điểm kinh doanh Công ty cổ phần dược phẩm ...,"1097B, Tỉnh lộ 43, Khu phố 2",Phường Bình Chiểu,Quận Thủ Đức,Ds. Trần Minh Trí,1131/CCHN-D-SYT-HCM,26/10/2017,Sở Y tế TP.HCM,8433/ĐKKDD-HCM,10113.0,GPP,10/07/2020,10/07/2023
4,Nhà thuốc,Bán lẻ thuốc thành phẩm,Nhà thuốc Nguyễn Nhi,"1B, Nguyễn Thượng Hiền",Phường 05,Quận Bình Thạnh,NaN,03135/HCM-CCHND,17/06/2015,Sở Y tế TP.HCM,5194/HCM-ĐKKDD,6121.0,GPP,13/10/2016,13/10/2019


### Kiểm tra dữ liệu bị thiếu

In [3]:
data.isna().sum()

LOAIHINHHN      114
PHAMVIHN         63
TENCOSO           0
DUONG             0
PHUONG           40
QUAN              2
CVXLCCHN       3775
SOCCHN            0
NGCAPCCHN       221
NOICAPCCHN        1
SODDK             2
SOGCN           735
GPP_GDP_GSP     735
NGAYCAPGCN     4606
NGAYHHGCN      4927
dtype: int64

Ta nhận thấy tồn tại dữ liệu bị thiếu ở các cột trong bảng dữ liệu, các cột `CVXLCCHN`, `SOGCN`, `GPP_GDP_GSP`, `NGAYCAPGCN`, `NGAYHHGCN` đều là các thuộc tính đặc thù, không quá quan trọng khi phân tích các thuộc tính đặc trưng khác nên tạm thời ta sẽ giữ lại các giá trị bị thiếu ở cột này mà chỉ đi xử lí ở các cột `LOAIHINHHN`, `PHAMVIHN`, `QUAN`

In [4]:
data = data.dropna(subset=['LOAIHINHHN','PHAMVIHN','QUAN'])
data.isna().sum()

LOAIHINHHN        0
PHAMVIHN          0
TENCOSO           0
DUONG             0
PHUONG           33
QUAN              0
CVXLCCHN       3758
SOCCHN            0
NGCAPCCHN       221
NOICAPCCHN        1
SODDK             2
SOGCN           729
GPP_GDP_GSP     729
NGAYCAPGCN     4599
NGAYHHGCN      4919
dtype: int64

In [5]:
data.shape

(16508, 15)

### Kiểm tra dữ liệu trùng lắp

In [6]:
print('Số lượng dòng dữ liệu trùng lắp:', data.duplicated().sum())

Số lượng dòng dữ liệu trùng lắp: 84


Ta nhận thấy có 84 dòng dữ liệu bị trùng lắp, ta tiến hành loại bỏ các dòng dữ liệu này

In [7]:
data.drop_duplicates(inplace=True)
print('Số lượng dòng dữ liệu sau khi loại trùng:', data.duplicated().sum())

Số lượng dòng dữ liệu sau khi loại trùng: 0


### Gom các quận trong thành phố Thủ Đức
Sau khi đã xử lí trùng, ta gom các quận Thủ Đức, 2 và 9 lại thành thành phố Thủ Đức.

In [8]:
districts = ['Quận Thủ Đức', 'Quận 2', 'Quận 9']

def normalize_address(address):
    if address in districts:
        return "TP. Thủ Đức"
    return address

data['QUAN'] = data['QUAN'].apply(normalize_address)

data['QUAN'].unique()

array(['Huyện Bình Chánh', 'TP. Thủ Đức', 'Quận Bình Thạnh', 'Quận 7',
       'Quận Gò Vấp', 'Quận 10', 'Huyện Hóc Môn', 'Huyện Củ Chi',
       'Quận 8', 'Quận 12', 'Quận Tân Phú', 'Huyện Cần Giờ', 'Quận 1',
       'Quận 11', 'Quận Bình Tân', 'Quận 4', 'Quận 5', 'Quận Tân Bình',
       'Quận 3', 'Huyện Nhà Bè', 'Quận 6', 'Quận Phú Nhuận'], dtype=object)

### Tinh chỉnh định dạng các quận

Tinh chỉnh các quận về giống định dạng trong tập tin liên quan đến cơ sở nhà thuốc để dễ dàng trực quan.

In [9]:
# Hàm giữ nguyên các quận có số, xóa "Quận" ở các quận không có số
def remove_quan_name(quan):
    if "Quận " in quan and not quan.split()[1].isdigit():
        return quan.replace("Quận ", "")
    return quan

# Áp dụng hàm trên cột QUAN
data['QUAN'] = data['QUAN'].apply(remove_quan_name)
data['QUAN'] = data['QUAN'].str.replace('Huyện ', '', regex=False)
data['QUAN'].unique()

array(['Bình Chánh', 'TP. Thủ Đức', 'Bình Thạnh', 'Quận 7', 'Gò Vấp',
       'Quận 10', 'Hóc Môn', 'Củ Chi', 'Quận 8', 'Quận 12', 'Tân Phú',
       'Cần Giờ', 'Quận 1', 'Quận 11', 'Bình Tân', 'Quận 4', 'Quận 5',
       'Tân Bình', 'Quận 3', 'Nhà Bè', 'Quận 6', 'Phú Nhuận'],
      dtype=object)

### Làm gọn loại hình nhà thuốc

In [10]:
data.sample(5)

,LOAIHINHHN,PHAMVIHN,TENCOSO,DUONG,PHUONG,QUAN,CVXLCCHN,SOCCHN,NGCAPCCHN,NOICAPCCHN,SODDK,SOGCN,GPP_GDP_GSP,NGAYCAPGCN,NGAYHHGCN
8320,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Hộ kinh doanh Nhà thuốc Minh Anh 1,"C4/2, Quốc lộ 1A, Ấp 3",Xã Tân Kiên,Bình Chánh,NaN,290/TTH-CCHND,31/07/2014,Sở Y tế Tỉnh Thừa Thiên - Huế,1255/ĐKKDD-HCM,5307.0,GPP,20/02/2024,20/02/2027
6690,Cơ sở bán buôn thuốc,"Bán buôn thuốc hóa dược, thuốc dược liệu, thuố...",Công ty TNHH Dược phầm Natra,"Số 6G16, Đường Trần Não, Khu phố 3",Phường An Khánh,TP. Thủ Đức,Ds. Bùi Hùng Cường,11619/CCHN-D-SYT-HCM,27/01/2023,Sở Y tế TP.HCM,10617/ĐKKDD-HCM,47.0,GDP,19/06/2023,10/01/2025
6407,Cơ sở bán buôn thuốc,"Bán buôn thuốc hóa dược, thuốc dược liệu, thuố...",Công ty TNHH Dược phẩm Minh Quân,"Quầy J10, 134/1, Tô Hiến Thành",Phường 15,Quận 10,Ds Tô Văn Minh,6055/HCM-CCHND,15/06/2016,Sở Y tế TP.HCM,4367/ĐKKDD-HCM,980.0,GDP,NaN,NaN
13021,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Hộ kinh doanh Nhà thuốc Thu Trà,"300/23/4, Nguyễn Văn Linh",Phường Bình Thuận,Quận 7,Ds. Nguyễn Trần Công Huân,4322/CCHN-D-SYT-HCM,24/04/2019,Sở Y tế TP.HCM,6943/ĐKKDD-HCM,9338.0,GPP,11/11/2022,11/11/2025
16257,Cơ sở bán lẻ thuốc: Nhà thuốc,"Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc ...",Công ty Cổ phần Dược phẩm FPT Long Châu - Địa ...,"Số 234/1, Đường Phạm Phú Thứ",Phường 04,Quận 6,Ds. Nguyễn Huy Hùng,7865/CCHN-D-SYT-HCM,15/06/2021,Sở Y tế TP.HCM,10641/ĐKKDD-HCM,11637.0,GPP,NaN,NaN


#### Kiểm tra các giá trị khác biệt trong cột `LOAIHINHHN`

In [11]:
data['LOAIHINHHN'].unique()

array(['Đại lý bán thuốc của doanh nghiệp',
       'Cơ sở bán lẻ thuốc: Nhà thuốc', 'Nhà thuốc',
       'Cơ sở bán buôn thuốc',
       'Cơ sở sản xuất thuốc đông y, thuốc từ dược liệu',
       'Doanh nghiệp bán buôn thuốc', 'Nhà thuốc bệnh viện công lập',
       'Cơ sở bán lẻ thuốc: Cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền',
       'Cơ sở bán lẻ thuốc: Nhà thuốc.', 'Cơ sở bán buôn thuốc.',
       'Doanh nghiệp xuất khẩu, nhập khẩu thuốc',
       'Cơ sở xuất nhập khẩu thuốc',
       'Doanh nghiệp sản xuất, bán buôn thuốc',
       'Cơ sở bán lẻ thuốc đông y, thuốc từ dược liệu',
       'Cơ sở bán lẻ thuốc: Nhà thuốc. Cơ sở bán lẻ thuốc: Cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền.',
       'Nhà thuốc của Doanh nghiệp', 'Cơ sở bán lẻ thuốc: Kệ thuốc',
       'Cơ sở bán buôn dược liệu, thuốc đông y, thuốc từ dược liệu',
       'Cơ sở bán buôn dược liệu',
       'Cơ sở bán buôn thuốc, nguyên liệu làm thuốc',
       'Doanh nghiệp sản xuất thuốc dùng

Ta nhận thấy dòng chữ "Cơ sở bán lẻ thuốc" là khá dư thừa trong chuỗi, ta tiến hành loại bỏ chuỗi này. Đồng thời có dấu '.' thừa ở cuối câu ta cũng sẽ loại bỏ

In [12]:
data['LOAIHINHHN'] = data['LOAIHINHHN'].str.replace('Cơ sở bán lẻ thuốc: ', '', regex=True)
data['LOAIHINHHN'] = data['LOAIHINHHN'].apply(lambda x: x.rstrip('.') if x.endswith('.') else x)
data['LOAIHINHHN'] = data['LOAIHINHHN'].str.lower()

#### Kiểm tra lại sau khi đã loại bỏ

In [13]:
unique_counts = data['LOAIHINHHN'].value_counts()

for value, count in unique_counts.items():
    print(f"'{value}': {count} lần")

'nhà thuốc': 13429 lần
'cơ sở bán buôn thuốc': 1976 lần
'cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền': 365 lần
'đại lý bán thuốc của doanh nghiệp': 152 lần
'doanh nghiệp bán buôn thuốc': 150 lần
'cơ sở bán lẻ thuốc đông y, thuốc từ dược liệu': 72 lần
'nhà thuốc của doanh nghiệp': 52 lần
'cơ sở bán buôn thuốc, nguyên liệu làm thuốc': 49 lần
'doanh nghiệp xuất khẩu, nhập khẩu thuốc': 39 lần
'cơ sở sản xuất thuốc đông y, thuốc từ dược liệu': 34 lần
'cơ sở bán buôn nguyên liệu làm thuốc': 21 lần
'cơ sở bán buôn dược liệu, thuốc đông y, thuốc từ dược liệu': 18 lần
'cơ sở chuyên bán lẻ thuốc dược liệu, thuốc cổ truyền': 14 lần
'nhà thuốc. cơ sở chuyên bán lẻ dược liệu, thuốc dược liệu, thuốc cổ truyền': 8 lần
'nhà thuốc bệnh viện công lập': 7 lần
'cơ sở chuyên bán lẻ dược liệu': 7 lần
'cơ sở bán buôn dược liệu': 5 lần
'kệ thuốc': 3 lần
'doanh nghiệp bán buôn dược liệu, thuốc đông y, thuốc từ dược liệu': 3 lần
'doanh nghiệp kinh doanh thuốc': 3 lần
'cơ sở xuất nhập khẩu th

In [14]:
data.isna().sum()

LOAIHINHHN        0
PHAMVIHN          0
TENCOSO           0
DUONG             0
PHUONG           33
QUAN              0
CVXLCCHN       3745
SOCCHN            0
NGCAPCCHN       221
NOICAPCCHN        1
SODDK             2
SOGCN           723
GPP_GDP_GSP     723
NGAYCAPGCN     4545
NGAYHHGCN      4869
dtype: int64

### Kiểm tra các giá trị trong cột `PHAMVIHN`

In [15]:
print(data['PHAMVIHN'].unique())
print('\nSố lượng giá trị khác biệt trong cột:',len(data['PHAMVIHN'].unique()))

['Bán lẻ thuốc thành phẩm thuộc danh mục thuốc thiết yếu (trừ thuốc gây nghiện, thuốc hướng tâm thần và tiền chất dùng làm thuốc và các thuốc kê đơn).'
 'Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm bảo quản điều kiện thường (Có bao gồm thuốc phải kiểm soát đặc biệt: Thuốc dạng phối hợp có chứa dược chất gây nghiện; Thuốc dạng phối hợp có chứa dược chất hướng thần; Thuốc dạng phối hợp có chứa tiền chất; Thuốc độc; Thuốc trong danh mục thuốc, dược chất thuộc danh mục chất bị cấm sử dụng trong một số ngành, lĩnh vực).'
 'Bán lẻ thuốc thành phẩm'
 'Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm (Có bao gồm thuốc phải kiểm soát đặc biệt: Thuốc dạng phối hợp có chứa dược chất gây nghiện; Thuốc dạng phối hợp có chứa dược chất hướng thần; thuốc dạng phối hợp có chứa tiền chất; Thuốc độc; Thuốc trong danh mục thuốc, dược chất thuộc danh mục chất bị cấm sử dụng trong một số ngành, lĩnh vực).'
 'Bán lẻ thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, si

Với số lượng khác biệt trong cột quá lớn, ta tạm không xét các mô tả chi tiết trong dấu ngoặc tròn để dễ dàng hơn trong phân tích. Đồng thời, các thông tin 'bán buôn' và 'bán lẻ' khá giống nhau, ta sẽ thay thế bằng 'bán' và loại bỏ các từ không quá quan trọng.

In [16]:
def remove_parentheses(text):
    parts = re.split(r"\(", text)
    if parts:
        return parts[0]
    return text

#Lọc các từ trong dấu ngoặc tròn
data['PHAMVIHN'] = data['PHAMVIHN'].apply(remove_parentheses)
data['PHAMVIHN'] = data['PHAMVIHN'].str.lower()

#Lọc các từ không mang nhiều ý nghĩa
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bán lẻ', 'bán')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bán buôn', 'bán')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bảo quản ở điều kiện thường ', '')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace('bảo quản điều kiện thường ', '')
data['PHAMVIHN'] = data['PHAMVIHN'].str.replace(' và', ',')
data['PHAMVIHN'] = data['PHAMVIHN'].apply(lambda x: x.rstrip('. ').rstrip() if 
                                         x.endswith('.') or x.endswith(' ') else x)

In [17]:
unique_counts = data['PHAMVIHN'].value_counts()
print(len(unique_counts))
for value, count in unique_counts.items():
    print(f"'{value}': {count} lần")

110
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm': 13914 lần
'bán thuốc thành phẩm': 980 lần
'bán thuốc trừ vắc xin': 480 lần
'bán thuốc dược liệu, thuốc cổ truyền': 201 lần
'bán thuốc thành phẩm thuộc danh mục thuốc thiết yếu': 152 lần
'bán dược liệu, thuốc dược liệu, thuốc cổ truyền': 132 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền': 117 lần
'bán thuốc đông y, thuốc từ dược liệu': 67 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, vắc xin, sinh phẩm': 47 lần
'bán dược liệu': 41 lần
'sản xuất thuốc đông y, thuốc từ dược liệu': 26 lần
'bán thuốc hóa dược': 21 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm bảo quản điều kiện thường': 19 lần
'bán dược liệu, vị thuốc cổ truyền': 13 lần
'xuất khẩu, nhập khẩu thuốc thành phẩm, vắc xin, sinh phẩm y tế': 12 lần
'xuất khẩu, nhập khẩu thuốc thành phẩm': 12 lần
'bán thuốc hóa dược, sinh phẩm': 10 lần
'bán thuốc hóa dược, thuốc dược liệu, thuốc cổ truyền, sinh phẩm bảo quản ở điều

### Kiểm tra cột `NGAYCAPGCN`

#### Ép kiểu dữ liệu về datetime

In [18]:
data['NGAYCAPGCN'] = pd.to_datetime(data['NGAYCAPGCN'], errors='coerce')

data['NGAYCAPGCN'].dtypes

/var/folders/p3/hp8lgp_x5lq310kt_5_c2fsc0000gn/T/ipykernel_45371/314129247.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['NGAYCAPGCN'] = pd.to_datetime(data['NGAYCAPGCN'], errors='coerce')


dtype('<M8[ns]')

#### Kiểm tra các năm 

In [19]:
unique_years = data['NGAYCAPGCN'].dt.year.unique()

# Hiển thị các năm
print(unique_years)

[  nan 2020. 2017. 2016. 2022. 2018. 2021. 2019. 2023. 2014. 2024. 2015.
 2013. 2026. 2012. 2011.]


Tồn tại các năm lớn hơn năm 2024 là vô lý, chứng tỏ dữ liệu bị nhập sai, ta tiến hành loại bỏ các dòng này

In [20]:
data = data[data['NGAYCAPGCN'].dt.year <= 2024]
unique_years = data['NGAYCAPGCN'].dt.year.unique()

# Hiển thị các năm
print(unique_years)

[2020 2017 2016 2022 2018 2021 2019 2023 2014 2024 2015 2013 2012 2011]


### Kiểm tra cột `NGAYCAPCCHN`

#### Ép kiểu dữ liệu về datetime

In [21]:
data['NGCAPCCHN'] = pd.to_datetime(data['NGCAPCCHN'], errors='coerce')

data['NGCAPCCHN'].dtypes

/var/folders/p3/hp8lgp_x5lq310kt_5_c2fsc0000gn/T/ipykernel_45371/3620005141.py:1: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data['NGCAPCCHN'] = pd.to_datetime(data['NGCAPCCHN'], errors='coerce')


dtype('<M8[ns]')

#### Kiểm tra các năm

In [22]:
unique_years = data['NGCAPCCHN'].dt.year.unique()

# Hiển thị các năm
print(unique_years)

[2019. 2016. 2017. 2015. 2013. 2022. 2021. 2020. 2014.   nan 2018. 2011.
 2024. 2023. 2012. 2010. 2009. 2026. 2007. 2031. 2002.]


Tồn tại các năm nan là vô lý, chứng tỏ dữ liệu bị nhập sai, ta tiến hành loại bỏ các dòng này

In [23]:
data = data[data['NGCAPCCHN'].dt.year <= 2024]
unique_years = data['NGCAPCCHN'].dt.year.unique()

# Hiển thị các năm
print(unique_years)

[2019 2016 2017 2015 2013 2022 2021 2020 2014 2018 2011 2024 2023 2012
 2010 2009 2007 2002]


### Loại bỏ các cột không mang nhiều ý nghĩa

Ta nhận thấy các cột `TENCOSO`, `SOCCHN`, `SODDK`, `SOGCN` là các cột không mang lại nhiều thông tin đáng giá để phân tích, ta tiến hành loại các cột này

In [24]:
data = data.drop(['TENCOSO', 'SOCCHN', 'SODDK', 'SOGCN'], axis=1)

Bộ dữ liệu sau khi đã tiền xử lí xong có dạng:

In [25]:
data.sample(7)

,LOAIHINHHN,PHAMVIHN,DUONG,PHUONG,QUAN,CVXLCCHN,NGCAPCCHN,NOICAPCCHN,GPP_GDP_GSP,NGAYCAPGCN,NGAYHHGCN
4154,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","11, Tam Bình, Khu phố 7",Phường Hiệp Bình Chánh,TP. Thủ Đức,Ds Lê Thị Thu Hòa,2014-03-28,Sở Y tế TP.HCM,GPP,2019-01-29,29/01/2022
1767,nhà thuốc,bán thuốc thành phẩm,"45, Huỳnh Văn Bánh",Phường 17,Phú Nhuận,Ds Trần Minh Trí,2015-05-25,Sở Y tế TP.HCM,GPP,2016-03-28,28/03/2019
15984,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","D6/55, Quốc lộ 1A, ấp 4",Xã Bình Chánh,Bình Chánh,Ds. Nguyễn Trần Công Huân,2021-03-26,Sở Y tế TP.HCM,GPP,2022-04-15,15/04/2025
14240,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","Số 91, đường Trần Não, khu phố 4",Phường An Khánh,TP. Thủ Đức,Ds. Nguyễn Trần Công Huân,2023-11-14,Sở Y tế TP.HCM,GPP,2023-12-29,29/12/2026
16095,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","318, Bắc Hải",Phường 06,Tân Bình,NaN,2021-02-02,Sở Y tế Tây Ninh,GPP,2022-02-22,22/02/2025
4763,nhà thuốc,"bán thuốc hóa dược, thuốc dược liệu, thuốc cổ ...","68, Tây Lân, Khu phố 7",Phường Bình Trị Đông A,Bình Tân,Ds. Nguyễn Huy Hùng,2020-09-28,Sở Y tế TP.HCM,GPP,2021-02-02,NaN
11620,nhà thuốc,bán thuốc thành phẩm,"13, Tuệ Tĩnh",Phường 13,Quận 11,Ds Phạm Văn Sơn,2014-01-15,Sở Y tế TP.HCM,GPP,2014-09-18,18/09/2017


### Lưu lại bộ dữ liệu

#### Lưu dưới dạng csv

In [26]:
data.to_csv('./Dataset/NhaThuoc_DaXuLy.csv', header=True, index=False)

#### Lưu dưới dạng excel

In [27]:
data.to_excel('./Dataset/NhaThuoc_DaXuLy.xlsx', header=True, index=False)